## Includes

In [1]:
import sys
import os
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 400)

import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config

config_info = {
    "canalyst_api_key":"",
   "s3_access_key_id": "",
    "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}

config = Config(config=config_info)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

A configuration file has been created for you in 
/Users/jedgore/canalyst/keys.json.


# Example: Redrive A Sector With A Macro "Hook"
## Workflow:
<li>Set switches including FRED series
    https://fred.stlouisfed.org
<li>Create ModelSet for target list
<li>Regress Key Drivers
<li>Apply filters to regression output from the control switches below
<li>Get new forecast values for second ticker based on individual regression formulas
<li>Redrive second model and display new values vs old

## Control Switches: 
### Change ticker as needed etc

In [2]:
target_list = ['CMA US',
              'WAL US',
              'FITB US',
              'CFR US',
              'HBAN US',
              'MTB US']

rsquared_cutoff = .5 # you might want a p-value filter in your version

return_series = "MO_RIS_EPS_WAD_Adj" 
fred_series = 'DGS10' #10 year treasury yield

forecast_quarter = '2022Q1'
forecast_period_name = 'Q1-2022'

## Use candas datareader to retrieve the FRED series
### Also calculate a year over year series for regression against growth rates

In [3]:
#candas datareader retrieves the FRED data
df_fred = cdr.get_fred_data([fred_series],config)

df_fred[fred_series + "_yoy"] = df_fred[fred_series].pct_change(periods=4)*100

df_fred['quarter'] = df_fred['quarter'].astype(str) #candas is all strings

df_fred = df_fred.dropna()

df_fred.tail()

,quarter,DGS10,DGS10_yoy
21,2021Q2,1.59,131.35
22,2021Q3,1.32,103.63
23,2021Q4,1.53,77.37
24,2022Q1,1.95,46.05
25,2022Q2,2.64,65.83


## Create ModelSet

In [4]:
target_modelset = cd.ModelSet(ticker_list = target_list, config=config)

## Check most recent quarter for our list

In [17]:
for ticker in target_list:
    print(f"{ticker} most recent quarter:")
    print(target_modelset.models[ticker].mrq())

CMA US most recent quarter:
  period_name period_end_date
0     Q4-2021      2021-12-31
WAL US most recent quarter:
  period_name period_end_date
0     Q4-2021      2021-12-31
FITB US most recent quarter:
  period_name period_end_date
0     Q4-2021      2021-12-31
CFR US most recent quarter:
  period_name period_end_date
0     Q4-2021      2021-12-31
HBAN US most recent quarter:
  period_name period_end_date
0     Q4-2021      2021-12-31
MTB US most recent quarter:
  period_name period_end_date
0     Q4-2021      2021-12-31


## Create a historical model frame for regression against Fred data

In [6]:
df_target = target_modelset.model_frame(is_historical=True,period_duration_type='fiscal_quarter')

## Get a list of key drivers for our target list

In [7]:
df_target_drivers = target_modelset.model_frame(is_driver=True,
                                       mrq=True)[['ticker','time_series_name','unit_type']]
df_target_drivers.head()

,ticker,time_series_name,unit_type
29229,CMA US,z_YCMC1Y0163_MO_OS_commercialloansavgbalancegrowth,percentage
60607,WAL US,z_2U1D9W0121_MO_OS_commercialloansavgbalancegrowth,percentage
90477,FITB US,z_EOA8J90125_MO_OS_totalcommercialloansandleasesavgbalancegrowth,percentage
106601,CFR US,z_ZOZFDY0111_MO_OS_totalsecuritiesavgbalancegrowth,percentage
135195,HBAN US,z_HI4B8P0124_MO_OS_totalsecuritiesavgbalancegrowth,percentage


## Regress Fred data against our modelset data

In [8]:
import statsmodels.formula.api as smf
import warnings
warnings.simplefilter("ignore")

df_out = []
df_models={}
for i,row in df_target_drivers.iterrows():
    
    fred_series_local = fred_series
    df1 = df_target.loc[df_target['time_series_name']==row['time_series_name']][['period_name_sorted','value','time_series_name']]
    if 'growth' in row['time_series_name']:
        fred_series_local = fred_series + "_yoy"
        df2 = df_fred[['quarter',fred_series_local]]
    else:
        df2 = df_fred[['quarter',fred_series_local]]
    
    df2.columns = ['quarter','value_x']
    df2 = pd.merge(df1,df2,how='inner',left_on='period_name_sorted',right_on='quarter')
    
    df2 = df2.dropna()
    
    if df2.shape[0] >0:
        model = smf.ols(formula="value ~ value_x", data=df2)
        rres = model.fit()
        d = {}
        d = {"ticker":row['ticker'],
            "source_driver":row['time_series_name'],
             "target_driver":fred_series_local,
            "intercept":[rres.params[0]],
             "n_periods":[df2.shape[0]],
            "rsquared":[rres.rsquared],
            "slope":[rres.params[1]],
            "pvalue":[rres.pvalues[1]]}
        df = pd.DataFrame(d)       
        df_out.append(df)

## Filter our regression output by our cutoff, add a count if we want to group and see number of key drivers 

In [9]:
df_regression = pd.concat(df_out)
df_regression = df_regression.sort_values('rsquared',ascending=False).dropna()
df_regression = df_regression[df_regression['rsquared']>rsquared_cutoff]
df_regression['count']=1
df_regression.sort_values(['ticker','rsquared'],ascending=False).head()

,ticker,source_driver,target_driver,intercept,n_periods,rsquared,slope,pvalue,count
0,WAL US,z_2U1D9W0121_MO_OS_Longtermdebtyield,DGS10,15.00,6,0.95,-6.41,0.00,1
0,WAL US,z_2U1D9W0121_MO_OS_Longtermdebtavgbalance,DGS10,1048384034.12,9,0.82,-409199133.86,0.00,1
0,WAL US,z_2U1D9W0121_MO_OS_commercialrealestateloansyield,DGS10,4.32,20,0.70,0.50,0.00,1
0,WAL US,z_2U1D9W0121_MO_OS_constructionandlanddevelopmentloansavgbalancegrowth,DGS10_yoy,15.31,20,0.69,0.18,0.00,1
0,WAL US,z_2U1D9W0121_MO_OS_Otheryield,DGS10,-0.66,20,0.65,1.05,0.00,1


## Group to get a look at how many key drivers we will be re-driving

In [11]:
df_regression.groupby('ticker').agg({'ticker':'first','rsquared':'mean','pvalue':'mean','count':'sum'}).sort_values('rsquared',ascending=False)

,ticker,rsquared,pvalue,count
ticker,,,,
WAL US,WAL US,0.65,0.00,11
HBAN US,HBAN US,0.62,0.00,10
MTB US,MTB US,0.60,0.00,7
FITB US,FITB US,0.59,0.00,12
CFR US,CFR US,0.58,0.00,5
CMA US,CMA US,0.57,0.00,9


## Filter our Fred data to the upcoming quarter

In [12]:
df_fred[df_fred['quarter']==forecast_quarter]

,quarter,DGS10,DGS10_yoy
24,2022Q1,1.95,46.05


## Some data munging to prepare the params dataframe for use in our fit() function

In [13]:
df_forecast = target_modelset.model_frame(period_name=forecast_period_name,is_driver=True,period_duration_type='fiscal_quarter')[['ticker',
                                                                                                                     'time_series_name',
                                                                                                                     'time_series_description',
                                                                                                                     'value',
                                                                                                                     'period_name',
                                                                                                                     'period_end_date',
                                                                                                                     'period_name_sorted']]
df_forecast = pd.merge(df_forecast,df_regression,how='inner',left_on=['ticker','time_series_name'],right_on=['ticker','source_driver'])
df_forecast = pd.merge(df_forecast,df_fred,how='inner',left_on='period_name_sorted',right_on='quarter')
dfilter = df_forecast['target_driver']==fred_series
df_values = df_forecast.where(dfilter).dropna()
df_values['new_value']=df_values[fred_series]
dfilter = df_forecast['target_driver']!=fred_series
df_values_yoy = df_forecast.where(dfilter).dropna()
df_values_yoy['new_value']=df_values_yoy[fred_series+"_yoy"]
df_forecast = pd.concat([df_values,df_values_yoy])
df_forecast['new_value']=df_forecast['new_value']*df_forecast['slope']+df_forecast['intercept']
df_forecast.head()

,ticker,time_series_name,time_series_description,value,period_name,period_end_date,period_name_sorted,source_driver,target_driver,intercept,n_periods,rsquared,slope,pvalue,count,quarter,DGS10,DGS10_yoy,new_value
8,HBAN US,z_HI4B8P0124_MO_OS_loansheldforsaleavgbalance,"Loans held for sale - avg. balance, mm",1525000000.00,Q1-2022,2022-03-31,2022Q1,z_HI4B8P0124_MO_OS_loansheldforsaleavgbalance,DGS10,1592567999.68,20.00,0.61,-353848957.98,0.00,1.00,2022Q1,1.95,46.05,902163024.72
9,CFR US,z_ZOZFDY0111_MO_OS_interestbearingdepositsyield,"Interest-bearing deposits yield, %",0.13,Q1-2022,2022-03-31,2022Q1,z_ZOZFDY0111_MO_OS_interestbearingdepositsyield,DGS10,-0.67,20.00,0.61,0.95,0.00,1.00,2022Q1,1.95,46.05,1.18
10,FITB US,z_EOA8J90125_MO_OS_Othershortterminvestmentsavgbalance,"Other short-term investments - avg. balance, mm",35000000000.00,Q1-2022,2022-03-31,2022Q1,z_EOA8J90125_MO_OS_Othershortterminvestmentsavgbalance,DGS10,40488541146.41,20.00,0.57,-14634778844.80,0.00,1.00,2022Q1,1.95,46.05,11934199261.65
11,CFR US,z_ZOZFDY0111_MO_OS_federalfundssoldandresellagreementsyield,"Federal funds sold and resell agreements yield, %",0.22,Q1-2022,2022-03-31,2022Q1,z_ZOZFDY0111_MO_OS_federalfundssoldandresellagreementsyield,DGS10,-0.52,20.00,0.62,0.91,0.00,1.00,2022Q1,1.95,46.05,1.26
12,MTB US,z_NEJWLB0175_MO_OS_interestbearingdepositsatbanksyield,"Interest-bearing deposits at banks yield, %",0.15,Q1-2022,2022-03-31,2022Q1,z_NEJWLB0175_MO_OS_interestbearingdepositsatbanksyield,DGS10,-0.61,20.00,0.61,0.90,0.00,1.00,2022Q1,1.95,46.05,1.13


## Create params dataframe

In [14]:
df_params = df_forecast[['ticker','period_name','time_series_name','time_series_description','value','new_value','period_end_date']]
df_params.head()

,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
8,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_loansheldforsaleavgbalance,"Loans held for sale - avg. balance, mm",1525000000.00,902163024.72,2022-03-31
9,CFR US,Q1-2022,z_ZOZFDY0111_MO_OS_interestbearingdepositsyield,"Interest-bearing deposits yield, %",0.13,1.18,2022-03-31
10,FITB US,Q1-2022,z_EOA8J90125_MO_OS_Othershortterminvestmentsavgbalance,"Other short-term investments - avg. balance, mm",35000000000.00,11934199261.65,2022-03-31
11,CFR US,Q1-2022,z_ZOZFDY0111_MO_OS_federalfundssoldandresellagreementsyield,"Federal funds sold and resell agreements yield, %",0.22,1.26,2022-03-31
12,MTB US,Q1-2022,z_NEJWLB0175_MO_OS_interestbearingdepositsatbanksyield,"Interest-bearing deposits at banks yield, %",0.15,1.13,2022-03-31


## Run the fit() function to generate new scenario

In [15]:
d_output=target_modelset.fit(df_params,return_series) #our fit function will return a link to scenario engine JSON for audit

MTB US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/NEJWLB0175/equity-models/FY2021.25/scenarios/TsiEjUqCVRGlt_gVNHHEjA/
HBAN US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/HI4B8P0124/equity-models/FY2021.27/scenarios/or1mthNKXh6kvAM3obtEpQ/
CFR US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/ZOZFDY0111/equity-models/FY2021.25/scenarios/7wvoaX2XXtycgfAe3JivNA/
CMA US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/YCMC1Y0163/equity-models/FY2021.26/scenarios/vXMfjhh1UgqEqjjbAbsxlg/
FITB US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/EOA8J90125/equity-models/FY2021.25/scenarios/nLqeNob5V0qalPHGEWmTEg/
WAL US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/2U1D9W0121/equity-models/FY2021.24/scenarios/0K0IIB1NWZa-pR0CXzjBtA/


## "Default" is the model current default, "Scenario" shows our changes

In [16]:
df_output = target_modelset.filter_summary(d_output,period_type='Q')
df_output[df_output['period_name']==forecast_period_name].sort_values('diff',ascending=False)

,ticker,period_name,time_series_name,time_series_description,default,scenario,diff
0,HBAN US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share (No Adjustments) - WAD,0.30188,0.5434580961,1.80025
0,FITB US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share (No Adjustments) - WAD,0.75773,1.1883503206,1.56831
0,MTB US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share (No Adjustments) - WAD,2.57122,3.4264602844,1.33262
0,CMA US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,1.24030,1.6448775828,1.32619
0,WAL US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share (No Adjustments) - WAD,2.43541,2.6902881619,1.10466
0,CFR US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,1.56654,1.5696655757,1.00199


## Loop over params dataframe to audit what we did

In [21]:
for ticker in target_list:
    print(ticker)
    display(df_params[df_params['ticker']==ticker])

CMA US


,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
20,CMA US,Q1-2022,z_YCMC1Y0163_MO_OS_commercialloansyield,"Commercial loans yield, %",3.67000,4.02370,2022-03-31
22,CMA US,Q1-2022,z_YCMC1Y0163_MO_OS_commercialmortgageloansyield,"Commercial mortgage loans yield, %",2.90000,3.94249,2022-03-31
23,CMA US,Q1-2022,z_YCMC1Y0163_MO_OS_internationalloansyield,"International loans yield, %",3.13000,4.19109,2022-03-31
26,CMA US,Q1-2022,z_YCMC1Y0163_MO_OS_interestbearingdepositswithbanksyield,"Interest-bearing deposits with banks yield, %",0.16000,1.14608,2022-03-31
35,CMA US,Q1-2022,z_YCMC1Y0163_MO_OS_shorttermborrowingsyield,"Short-term borrowings yield, %",0.05000,1.08894,2022-03-31
49,CMA US,Q1-2022,MO_ACL_ALLRatio,"Allowance for Loan Losses as % of Total Loans, %",1.25000,1.40788,2022-03-31
53,CMA US,Q1-2022,z_YCMC1Y0163_MO_RC_RiskweightedAssetsasaofAvgInterestEarningAssets,"Risk-weighted Assets as a % of Avg. Interest Earning Assets, %",91.00000,95.61355,2022-03-31
0,CMA US,Q1-2022,z_YCMC1Y0163_MO_OS_commercialloansavgbalancegrowth,"Commercial loans - avg. balance growth, %",5.00000,-3.93048,2022-03-31
45,CMA US,Q1-2022,z_YCMC1Y0163_MO_OS_commerciallendingfeegrowth,"Commercial lending fee growth, %",5.50000,16.09816,2022-03-31


WAL US


,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
18,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_commercialloansyield,"Commercial loans yield, %",4.73000,5.19366,2022-03-31
19,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_commercialrealestateloansyield,"Commercial real estate loans yield, %",4.75000,5.30110,2022-03-31
21,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_residentialrealestateloansyield,"Residential real estate loans yield, %",3.60000,4.30611,2022-03-31
24,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_totalsecuritiesyield,"Total securities yield, %",2.90000,2.97546,2022-03-31
25,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_Otheryield,"Other yield, %",0.24000,1.38497,2022-03-31
31,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_Longtermdebtavgbalance,"Long-term debt - avg. balance, mm",0.00000,249983724.08104,2022-03-31
32,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_shorttermborrowingsyield,"Short-term borrowings yield, %",0.18000,1.19576,2022-03-31
33,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_Longtermdebtyield,"Long-term debt yield, %",0.00000,2.48676,2022-03-31
34,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_Qualifyingdebtyield,"Qualifying debt yield, %",4.39000,5.21658,2022-03-31
4,WAL US,Q1-2022,z_2U1D9W0121_MO_OS_constructionandlanddevelopmentloansavgbalancegrowth,"Construction and land development loans - avg. balance growth, %",20.00000,23.80028,2022-03-31


FITB US


...

,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
9,CFR US,Q1-2022,z_ZOZFDY0111_MO_OS_interestbearingdepositsyield,"Interest-bearing deposits yield, %",0.13000,1.18069,2022-03-31
11,CFR US,Q1-2022,z_ZOZFDY0111_MO_OS_federalfundssoldandresellagreementsyield,"Federal funds sold and resell agreements yield, %",0.22000,1.25528,2022-03-31
44,CFR US,Q1-2022,MO_ACL_ALLRatio,"Allowance for Loan Losses as % of Total Loans, %",1.50000,1.40788,2022-03-31
3,CFR US,Q1-2022,z_ZOZFDY0111_MO_OS_netofunearneddiscountsloansavgbalancegrowth,"Net of unearned discounts loans - avg. balance growth, %",-12.00000,3.45700,2022-03-31
36,CFR US,Q1-2022,z_ZOZFDY0111_MO_OS_trustandinvestmentmanagementfeesgrowth,"Trust and investment management fees growth, %",5.00000,10.96923,2022-03-31


HBAN US


,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
8,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_loansheldforsaleavgbalance,"Loans held for sale - avg. balance, mm",1525000000.00000,902163024.72330,2022-03-31
13,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_interestbearingdepositsinbanksyield,"Interest-bearing deposits in banks yield, %",0.10000,1.22849,2022-03-31
14,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_InterestbearingdepositsinFederalReserveBankyield,"Interest-bearing deposits in Federal Reserve Bank yield, %",0.30000,1.39955,2022-03-31
15,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_loansheldforsaleyield,"Loans held for sale yield, %",3.50000,3.54813,2022-03-31
17,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_commercialloansyield,"Commercial loans yield, %",4.50000,4.15456,2022-03-31
43,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_ServiceChargesasofAvgDepositsAnnualized,"Service Charges as % of Avg. Deposits (Annualized), %",0.40000,0.54639,2022-03-31
47,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_Gainonsalemargin,"Gain on sale margin, %",3.50000,2.85091,2022-03-31
50,HBAN US,Q1-2022,MO_ACL_ALLRatio,"Allowance for Loan Losses as % of Total Loans, %",1.90000,1.40788,2022-03-31
39,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_cardsandpaymentprocessingrevenuegrowth,"Cards and payment processing revenue growth, %",40.00000,23.35518,2022-03-31
48,HBAN US,Q1-2022,z_HI4B8P0124_MO_OS_Thirdpartymortgageloansgrowth,"Third party mortgage loans growth, %",30.00000,15.73202,2022-03-31


MTB US


,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
12,MTB US,Q1-2022,z_NEJWLB0175_MO_OS_interestbearingdepositsatbanksyield,"Interest-bearing deposits at banks yield, %",0.15000,1.13403,2022-03-31
28,MTB US,Q1-2022,z_NEJWLB0175_MO_OS_longtermborrowingsyield,"Long-term borrowings yield, %",1.75000,2.36450,2022-03-31
40,MTB US,Q1-2022,z_NEJWLB0175_MO_OS_TrustMERAnnualized,"Trust MER (Annualized), %",0.44000,0.56835,2022-03-31
41,MTB US,Q1-2022,z_NEJWLB0175_MO_OS_ServicechargesondepositaccountsasofAvgDepositsAnnualized,"Service charges on deposit accounts as % of Avg. Deposits (Annualized), %",0.55000,0.64031,2022-03-31
51,MTB US,Q1-2022,z_NEJWLB0175_MO_DS_NewshareIssuanceBuybacks,New share Issuance (Buybacks),0.00000,-257721505.43046,2022-03-31
2,MTB US,Q1-2022,z_NEJWLB0175_MO_OS_totalloansandleasesavgbalancegrowth,"Total loans and leases - avg. balance growth, %",5.00000,-0.32356,2022-03-31
37,MTB US,Q1-2022,z_NEJWLB0175_MO_OS_mortgagebankingrevenuesgrowth,"Mortgage banking revenues growth, %",1.00000,1.84980,2022-03-31
